## Instructions
* Read each cell and implement the **TODOs** sequentially. The markdown/text cells also contain instructions which you need to follow to get the whole notebook working.
* Do not change the variable names unless the instructor allows you to.
* Do not delete the **TODO** comment blocks.
* Aside from the TODOs, there will be questions embedded in the notebook and a cell for you to provide your answer (denoted with A:). Answer all the markdown/text cells with **"A: "** on them. 
* You are expected to search how to some functions work on the Internet or via the docs. 
* You may add new cells for "scrap work".
* The notebooks will undergo a "Restart and Run All" command, so make sure that your code is working properly.
* You are expected to understand the data set loading and processing separately from this class.
* You may not reproduce this notebook or share them to anyone.

Place your answers to the questions directly inline on the same cell as **A:**

For example:

<span style='color:red'>**Question 00:**</span> What is your favorite ice cream flavor?

<span style='color:red'>**A00:**</span> My favorite flavor ice cream flavor is pistachio.

# Assignment 1.4 - Knowledge Based AI Notebook 
In this notebook, you will be experimenting on Bayesian networks on the Dutch census dataset and implementing a simple sudoku solver using logic rules.

# Bayesian Networks

For this case study, we are going to look at Bayesian networks. 
A Bayesian network is a directed acyclic graph that represent the dependency structure of a set of variables. 
We can use this model to see how changes in one variable affect one another and perform inference queries.

In [1]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import matplotlib.pyplot as plt
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
import pyAgrum.causal as csl
import pyAgrum.causal.notebook as cslnb
import scipy.stats
from sklearn import preprocessing

Graphviz is not installed. 
  Please install this program in order to visualize graphical models in pyAgrum. 
  See https://graphviz.org/download/


Let's start first with loading the dataset that we are gonna be using.

### The Dutch Virtual Census of 2001 dataset
https://microdata.worldbank.org/index.php/catalog/2102/study-description

The dataset includes 60,420 samples where each sample is described by 12 variables. The table below shows an overview of the variables.
Note that this dataset is a pre-processed version of the original where the range of values have already been discretized (converted to a small set of categories) for simplicity.

| Attributes           | Type        | Values                         | Description                                                    |
| -------------------- | ----------- | ------------------------------ | -------------------------------------------------------------- |
| sex                  | Binary      | {0: Female, 1: Male}                  | The biological sex of the person                               |
| age                  | Categorical | 12                             | The age group of the person (0–4, 5–9, etc.)                   |
| household_position   | Categorical | 8                              | The relationship to household head (spouse, child, etc.)       |
| household_size       | Categorical | 6                              | The size of the household the person belongs to                |
| prev_residence_place | Binary      | {0: non-Netherlands, 1: Netherlands} | The place of the person's residence prior to the Census        |
| citizenship          | Categorical | 3                              | The person's citizenship status                                |
| country_birth        | Categorical | 3                              | Whether the person was born in the Netherlands or elsewhere    |
| edu_level            | Categorical | 6                              | The person's level of educational attainment                   |
| economic_status      | Categorical | 3                              | The person's economic status (class of worker)                 |
| cur_eco_activity     | Categorical | 12                             | The current economic activity                                  |
| marital_status       | Categorical | 4                              | The person's current marital status according to law or custom |
| occupation           | Binary      | {0: low-level, 1: high-level}                          |  The person's occupation (0: low-level, 1: high-level)         |

In [2]:
df = pd.read_csv("dutch_preprocessed.csv")
df.head()

,sex,age,household_position,household_size,prev_residence_place,citizenship,country_birth,edu_level,economic_status,cur_eco_activity,marital_status,occupation
0,1,6,3,1,1,1,1,5,0,7,1,1
1,0,10,2,2,1,1,1,2,0,1,2,0
2,1,8,2,2,1,1,1,2,0,1,2,1
3,1,12,1,1,1,1,1,1,0,9,2,0
4,0,4,0,3,1,1,1,2,0,10,1,0


## Defining graph structure

One way to incorporate domain knowledge is by defining structure of this dependency graph.
The structure of the graph would impose certain constraints and determine how the different variables affect one another.

Since we do not know the dependency graph for this dataset, we can try to first infer it from data.

<span style='color:red'>**TODO-01:**</span> The pyAgrum library has a tutorial below on how to learn the structure from data. Follow this tutorial and learn a plausible structure for this dutch census dataset.

https://pyagrum.readthedocs.io/en/1.15.1/notebooks/31-Learning_structuralLearning.html

Don't forget to visualize the graph. You can add as many cells below as necessary.

In [ ]:
# Learning the structure
learner = gum.BNLearner(df)
learner.useLocalSearchWithTabuList()
bn = learner.learnBN()

# Visualize the graph
gnb.showBN(bn)


<span style='color:red'>**Question 01:**</span> Do the relationships learned only from the data make sense? Why or why not? Explain your answer.

<span style='color:red'>**A01:**</span>

<span style='color:red'>**Question 02:**</span> If you were going to modify certain relationships, which ones would you change? How will your graph look like?


<span style='color:red'>**A02:**</span>

<span style='color:red'>**TODO-02:**</span> Incorporate the modifications to the graph to align with your domain knowledge.

You can incorporate this in the graph learning by adding mandatory arcs.

For example, "I know that smoking causes cancer"

``learner.addMandatoryArc("smoking","lung_cancer") # smoking->lung_cancer``

Ofcourse there's its opposite where you add forbidden arcs. `learner.addForbiddenArc()`

You can optionally define the entire graph yourself using `addArc` function.

Don't forget to visualize the graph. You can add as many cells below as necessary.

## Learning the parameters
Now that you have defined a graph, we need to learn the parameters. In this case, these parameters would correspond to the conditional probabilities for each node given its parents.

<span style='color:red'>**TODO-03:**</span> Follow the tutorial on learning parameters from a database (in this case a csv file) and learn the parameters for the dutch census dataset.
                                                                                                                       
https://pyagrum.readthedocs.io/en/1.15.1/notebooks/31-Learning_structuralLearning.html#Parameters-learning-from-the-database

Visualize some of the learned conditional probability tables (cpt). (Minimum of 2)

## Inference queries

Now that we have a learned model, we can answer inference queries.
Look at the tutorial on causality and counterfactuals from pyAgrum to get an idea on how this is done. 

https://pyagrum.readthedocs.io/en/1.15.1/notebooks/65-Causality_Counterfactual.html

<span style='color:red'>**TODO-04:**</span> Perform some counterfactual inference queries using the learned model above. Show and interpret at least two examples. You are free to choose which variables you want to investigate. 

<span style='color:red'>**Question 03:**</span> Using the observations from the results of the previous cells, can you explain why standard deep learning models are not suitable for counterfactual reasoning?

<span style='color:red'>**A03:**</span>

# Solving Sudoku by specifying logic rules

For this we will be using the z3-solver, which you can install using the command below.

``pip install z3-solver``

https://z3prover.github.io/papers/programmingz3.html#sec-terms-and-formulas

### Background brief tutorial with z3 solver

In [ ]:
import matplotlib as mpl
import numpy as np
from z3 import *

To use the z3 solver, we need to first instantiate a solver instance.

In [ ]:
solver = Solver()

Next, the only thing we need to do is to add the logical rules that would serve as the constraints in searching for a solution. z3 solver will handle the rest.

To do this, we need to first define our atoms. For our case, we would only be dealing with integer atoms and you can define them as follows.

In [ ]:
A = Int("A")
B = Int("B")

You can add rules using the `solver.add()` function. You can pass in expressions as a parameter.

In [ ]:
solver.add(A > 0)

You could also use `And()` `Or()` operators. By default, all the rules you add are combined by `And`.

In [ ]:
solver.add(Or(B < 0, B > 5))

You can also use arithmetic operations.

In [ ]:
solver.add(A + B == 2)

As well as `Distinct()` function.

In [ ]:
solver.add(Distinct(A,B))

In [ ]:
print(solver.check() == sat) # sat is a special variable which the solver will return if the constraints are satisfiable 
print(solver.model()) # we can get a solution through the solver.model()

## Sudoku solver

Now we are ready to work on the sudoku solver.

In [ ]:
rows = 'ABCDEFGHI'
cols = '123456789'
sudoku_boxes = np.array([[Int("{}{}".format(r, c)) for c in cols] for r in rows])

In this case, the atoms are in the matrix `sudoku_boxes`. Note that it is currently in a numpy array for easy indexing. However, you will need to convert it to lists later on since some of the functions would expect an ordinary python list, which you can easily do by using the `.tolist()` function.

In [ ]:
sudoku_boxes

In [ ]:
sudoku_boxes.tolist()

Another useful function is the `chain` function from `itertools` library which flatten nested lists; `chain(*[[a, b], [c, d], ...]) == [a, b, c, d, ...]`

In [ ]:
from itertools import chain
print(list(chain(*sudoku_boxes)))

Another useful trick is the `*` operator in python. Particularly, it allows you to unpack a list and individually pass it as a parameter.

For example,

`letters = ["a", "b", "c", "d"]`

`print(*letters)` is equivalent to `print("a","b","c","d")`



In [ ]:
letters = ["a", "b", "c", "d"]
print(letters)

In [ ]:
print(*letters)

In [ ]:
print("a","b","c","d")

Let's instantiate the sudoku solver!

In [ ]:
sudoku_solver = Solver()

In [ ]:
###############################################################
# TODO-05: Add the constraint that dictates the range of      #
# values that each atom (each cell) in the sudoku can         #
# have.                                                       #
###############################################################



###############################################################
#                    End of your code.                        #
###############################################################

In [ ]:
###############################################################
# TODO-06: Add the constraint that each row has a distinct    #
# value.                                                      #
###############################################################


###############################################################
#                    End of your code.                        #
###############################################################

In [ ]:
###############################################################
# TODO-07: Add the constraint that each column has a distinct #
# value.                                                      #
###############################################################


###############################################################
#                    End of your code.                        #
###############################################################

In [ ]:
###############################################################
# TODO-08: Add the constraint that each 3x3 block has a       #
# distinct value.                                             #
###############################################################


###############################################################
#                    End of your code.                        #
###############################################################

Now that we have encoded the general rules of sudoku, we need to account for the particular constraints in an instance of a sudoku puzzle. Particularly, the constraints that certain boxes already have a pre-determined value as specified by the instance of the sudoku puzzle.

We define below an instance of a sudoku board configuration. The 0 value indicates that the box does not have an assigned value and it will have to be determined by the solver.

In [ ]:
board = ((0,0,0,2,0,7,1,3,0),
        (5,4,0,0,6,0,0,8,0),
        (0,1,0,9,0,0,0,5,2),
        (9,5,0,0,8,2,0,0,6),
        (0,0,6,0,0,0,7,0,0),
        (3,0,0,5,9,0,0,4,8),
        (8,2,0,0,0,4,0,7,0),
        (0,3,0,0,2,0,0,6,9),
        (0,6,5,1,0,9,0,0,0))


In [ ]:
###############################################################
# TODO-09: Add the constraint specifying the given values of  #
# the boxes as determined by the given sudoku board           #
# configuration.                                              #
###############################################################


###############################################################
#                    End of your code.                        #
###############################################################

The code below checks for the satisfiability and displays the found solution.

In [ ]:
assert sudoku_solver.check() == sat, "The solver didn't find a solution. Double check your constraints."
for row, _boxes in enumerate(sudoku_boxes):
    if row and row % 3 == 0:
        print('-'*9+"|"+'-'*9+"|"+'-'*9)
    for col, box in enumerate(_boxes):
        if col and col % 3 == 0:
            print('|', end='')
        print(' {} '.format(sudoku_solver.model()[box]), end='')
    print()

<span style='color:red'>**Question 04:**</span> Discuss the advantages and disadvantages of this approach as compared with a machine learning sudoku solver in terms of scalability on space and time complexity as well as accuracy of the solutions.

<span style='color:red'>**A04:**</span>

<span style='color:red'>**Question:**</span> How much time did it take you to answer this notebook?

<span style='color:red'>**A:**</span>